In [1]:
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.ML.LightGbm"
    
//Install Daany packages
#r "nuget:Daany.DataFrame"
#r "nuget:Daany.DataFrame.Ext"
#r "nuget:Daany.Stat"
    
//Install XPlot package
#r "nuget:XPlot.Plotly"

SyntaxError: invalid syntax (<ipython-input-1-a94f6d6beef0>, line 4)

# Building Predictive Maintenance Model Using ML.NET

## Summary
This C# notebook is a continuation from the previous blog post [Predictive Maintenance on .NET Platform](https://bhrnjica.net/2019/12/04/predictive-maintenance-on-net-platform/).

The notebook is completely implemented on .NET platform using `C# Jupyter Notebook` and `Daany` - C# data analytics library. There are small differences between this notebook and the notebooks at the official azure gallery portal, but in most cases, the code follows the steps defined there.

The notebook shows how to use `.NET Jupyter Notebook` with `Daany.DataFrame` and `ML.NET` in order to prepare the data and build the Predictive Maintenance Model on .NET platform.  

## Description

In the previous post, we analyzed 5 data sets with information about `telemetry`, `data`, `errors` and `maintenance` as well as `failure` for 100 machines. The data were transformed and analyzed in order to create the final data set for building a machine learning model for Predictive maintenance.

Once we created all features from the data sets, as a final step we created the label column so that it describes if a certain machine will fail in the next 24 hours due to failure a `component1`, `component2`, `component3`, `component4` or it will continue to work.
.
In this part, we are going to perform a part of the machine learning task and start training a machine learning model for predicting if a certain machine will fail in the next 24 hours due to failure, or it will be in functioning normal in that time period. 

The model which we are going to build is multiclass classification model sice it has 5 values to predict: 
- `component1`,
- `component2`, 
- `component3`, 
- `component4` or
- `none` - means it will continue to work.

# ML.NET framework as librabry for training

In order to train the model, we are going to use ML.NET - Microsoft open source framerowkr for Machine Leanirng on .NET Platform. 
First we need to put some preparation codes like:
- Required Nuget packages,
- Set of using statements and code for formatting the output:

At the beggining of this notebook, we installed the several NugetPackages in order to complete this notebook. The following code shows using statements, and method for formatting the data from the DataFrame.

In [2]:
//using Microsoft.ML.Data;
using XPlot.Plotly;
using System;
using System.Collections.Generic;
using System.Drawing;
using System.Linq;

//
using Microsoft.ML;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms;
using Microsoft.ML.Trainers.LightGbm;
//
using Daany;
using Daany.Ext;
//DataFrame formatter
using Microsoft.AspNetCore.Html;
Formatter<DataFrame>.Register((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c)));
    
    //renders the rows
    var rows = new List<List<IHtmlContent>>();
    var take = 20;
    
    //
    for (var i = 0; i < Math.Min(take, df.RowCount()); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(df.Index[i]));
        foreach (var obj in df[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }
    
    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));
    
    writer.Write(t);
}, "text/html");

Once we install the Nuget packages and define using statements we are going to define a class we need to create an ML.NET pipeline.

The class `PrMaintenanceClass` - contains the features (properties) we build in the previous post. We need them to define features in the ML.NET pipeline. The second class we defined is `PrMaintenancePrediction` we used for prediction and model evaluation.

In [3]:
class PrMaintenancePrediction
{
    [ColumnName("PredictedLabel")]
    public string failure { get; set; }
}
class PrMaintenanceClass
{
    public DateTime datetime { get; set; }
    public int machineID { get; set; }
    public float voltmean_3hrs { get; set; }
    public float rotatemean_3hrs { get; set; }
    public float pressuremean_3hrs { get; set; }
    public float vibrationmean_3hrs { get; set; }
    public float voltstd_3hrs { get; set; }
    public float rotatestd_3hrs { get; set; }
    public float pressurestd_3hrs { get; set; }
    public float vibrationstd_3hrs { get; set; }
    public float voltmean_24hrs { get; set; }
    public float rotatemean_24hrs { get; set; }
    public float pressuremean_24hrs { get; set; }
    public float vibrationmean_24hrs { get; set; }
    public float voltstd_24hrs { get; set; }
    public float rotatestd_24hrs { get; set; }
    public float pressurestd_24hrs { get; set; }
    public float vibrationstd_24hrs { get; set; }
    public float error1count { get; set; }
    public float error2count { get; set; }
    public float error3count { get; set; }
    public float error4count { get; set; }
    public float error5count { get; set; }
    public float sincelastcomp1 { get; set; }
    public float sincelastcomp2 { get; set; }
    public float sincelastcomp3 { get; set; }
    public float sincelastcomp4 { get; set; }
    public string model { get; set; }
    public float age { get; set; }
    public string failure { get; set; }
}

Now that we have defined a class type, we are going to implement the pipleine for this ml model.First, we create `MLContext` with constant seed, so that the model can be reproduced by any user running this notebook. Then we load the data and split the data into train and test set.

In [4]:
MLContext mlContext= new MLContext(seed:88888);
var strPath="data/final_dataFrame.csv";
var mlDF= DataFrame.FromCsv(strPath);
//
//split data frame on training and testing part
//split at 2015-08-01 00:00:00, to train on the first 8 months and test on last 4 months
var trainDF = mlDF.Filter("datetime", new DateTime(2015, 08, 1, 1, 0, 0), FilterOperator.LessOrEqual);
var testDF = mlDF.Filter("datetime", new DateTime(2015, 08, 1, 1, 0, 0), FilterOperator.Greather);

In [5]:
trainDF.Describe()

Count,168876,168876,168876,168876,168876,168876,168876,168876,168876,168876,168876,168876,168876,168876,168876,168876,168876,168876,168876,168876,168876,168876,168876,168876,168876,168876,168876
Unique,100,162187,164744,163901,163802,162187,164744,163901,163802,151923,158491,155825,156039,166584,166607,166126,163498,3,3,3,3,3,3166,2792,2968,2680,20
Top,22,175.460434,477.667084,103.714561,37.233704,175.460434,477.667084,103.714561,37.233704,172.87442,454.052643,99.487488,40.20649,18.003925,53.94923,8.729134,5.07115,0,0,0,0,0,4.958333,0.958333,0.083333,0.083333,14
Freq,1703,4,4,4,4,4,4,4,4,5,4,5,5,3,3,3,4,164132,164458,164826,165640,167118,398,443,409,409,23682
Mean,50.492231,170.799561,446.600098,100.815468,40.394905,170.799561,446.600098,100.815468,40.394905,170.805573,446.601318,100.81736,40.395767,14.90595,49.911221,10.036919,4.998013,0.028441,0.026605,0.024124,0.01931,0.010493,60.348289,59.665043,61.300648,60.179634,11.331788
Std,28.872004,9.503836,33.04018,7.368439,3.480648,9.503836,33.04018,7.368439,3.480648,4.760393,18.012669,4.692562,2.077442,2.269454,7.690305,1.702571,0.803144,0.168318,0.163664,0.154359,0.138684,0.102707,69.981432,69.818737,68.87872,65.756215,5.827269
Min,1,125.532501,219.152267,72.118637,26.569635,125.532501,219.152267,72.118637,26.569635,156.284729,267.938446,90.352638,35.253277,6.63169,22.349758,4.433239,2.107598,0,0,0,0,0,0.083333,0.083333,0.083333,0.083333,0
25%,25,164.480766,427.591972,96.217316,38.156631,164.480766,427.591972,96.217316,38.156631,168.073498,441.476715,98.654911,39.366099,13.329696,44.635914,8.919842,4.45405,0,0,0,0,0,13.458333,12.208333,13.458333,13.333333,7
Median,50,170.446716,448.414551,100.204285,40.152775,170.446716,448.414551,100.204285,40.152775,170.229919,449.176208,100.083374,40.077965,14.845225,49.580578,9.913457,4.952832,0,0,0,0,0,34.208332,31.083334,34.958332,34.208332,12
75%,75,176.613708,468.372093,104.362736,42.22999,176.613708,468.372093,104.362736,42.22999,172.474007,456.397896,101.588448,40.835839,16.396019,54.748691,10.968109,5.483441,0,0,0,0,0,76.958336,75.833336,79.083336,82.489586,16
Max,100,241.420715,584.83075,162.29805,69.311325,241.420715,584.83075,162.29805,69.311325,220.569061,493.554901,152.314606,61.259308,27.539467,100.707497,24.078493,11.234724,2,2,2,2,2,395.708344,348.958344,370.958344,334.958344,20


In [6]:
testDF.Describe()

Count,122752,122752,122752,122752,122752,122752,122752,122752,122752,122752,122752,122752,122752,122752,122752,122752,122752,122752,122752,122752,122752,122752,122752,122752,122752,122752,122752
Unique,100,119141,120591,119995,120053,119141,120591,119995,120053,113330,116904,115881,115826,121427,121494,121206,119781,3,3,3,3,3,2588,1800,1920,2994,20
Top,13,174.895172,471.315857,100.375046,39.443283,174.895172,471.315857,100.375046,39.443283,170.031601,446.008667,101.329124,40.84037,14.206243,45.504265,10.355402,4.527656,0,0,0,0,0,0.833333,2.833333,14.833333,14.833333,14
Freq,1242,4,4,4,3,4,4,4,3,4,4,4,4,3,3,3,3,119398,119167,119988,120138,121495,305,327,307,312,17212
Mean,50.496318,170.756744,446.49057,100.932724,40.384457,170.756744,446.49057,100.932724,40.384457,170.750656,446.489044,100.927963,40.384068,14.93679,50.001114,10.060551,5.007632,0.027788,0.029433,0.022941,0.021474,0.010273,44.073059,40.16396,40.704807,44.921112,11.333632
Std,28.865789,9.515308,33.404365,7.509693,3.491901,9.515308,33.404365,7.509693,3.491901,4.712856,18.473802,4.860883,2.07145,2.248045,7.690313,1.725013,0.794275,0.167166,0.170363,0.152518,0.146191,0.101156,48.804064,37.297347,37.872459,48.634826,5.826362
Min,1,132.588959,211.811188,74.875999,27.682785,132.588959,211.811188,74.875999,27.682785,157.193085,267.008148,91.159561,36.023071,6.502827,19.83952,4.445923,2.275665,0,0,0,0,0,0.083333,0.083333,0.083333,0.083333,0
25%,25,164.39756,427.401917,96.265844,38.13948,164.39756,427.401917,96.265844,38.13948,168.064827,441.647293,98.692015,39.345946,13.39748,44.716764,8.930172,4.471256,0,0,0,0,0,13.208333,11.958333,12.833333,12.708333,7
Median,50,170.425323,448.273499,100.27816,40.144547,170.425323,448.273499,100.27816,40.144547,170.188446,449.207001,100.119148,40.062256,14.870501,49.668045,9.935278,4.964489,0,0,0,0,0,30.958334,28.458334,29.583334,30.458334,12
75%,76,176.617683,468.522041,104.471903,42.23519,176.617683,468.522041,104.471903,42.23519,172.434006,456.321007,101.653372,40.830544,16.396651,54.883877,11.000475,5.48524,0,0,0,0,0,59.958332,57.583332,56.833332,60.083332,16
Max,100,234.51091,586.682861,162.309662,65.522766,234.51091,586.682861,162.309662,65.522766,218.996292,499.288788,152.660904,61.85284,27.914051,105.326447,28.867737,12.609035,2,2,2,2,2,491.958344,224.958328,239.958328,394.958344,20


Once we have data into application memory, we can prepare the ML.NET pipeline. The pipeline consists of data transformation from the `Daany.DataFrame` type into collection `IDataView`. For this task, the `LoadFromEnumerable` method is used.

In [7]:
//Load daany:DataFrame into ML.NET pipeline
public static IDataView loadFromDataFrame(MLContext mlContext,Daany.DataFrame df)
{
    IDataView dataView = mlContext.Data.LoadFromEnumerable<PrMaintenanceClass>(df.GetEnumerator<PrMaintenanceClass>(oRow =>
    {
        //convert row object array into PrManitenance row
        var ooRow = oRow;
        var prRow = new PrMaintenanceClass();
        prRow.datetime = (DateTime)ooRow["datetime"];
        prRow.machineID = (int)ooRow["machineID"];
        prRow.voltmean_3hrs = Convert.ToSingle(ooRow["voltmean_3hrs"]);
        prRow.rotatemean_3hrs = Convert.ToSingle(ooRow["rotatemean_3hrs"]);
        prRow.pressuremean_3hrs = Convert.ToSingle(ooRow["pressuremean_3hrs"]);
        prRow.vibrationmean_3hrs = Convert.ToSingle(ooRow["vibrationmean_3hrs"]);
        prRow.voltstd_3hrs = Convert.ToSingle(ooRow["voltsd_3hrs"]);
        prRow.rotatestd_3hrs = Convert.ToSingle(ooRow["rotatesd_3hrs"]);
        prRow.pressurestd_3hrs = Convert.ToSingle(ooRow["pressuresd_3hrs"]);
        prRow.vibrationstd_3hrs = Convert.ToSingle(ooRow["vibrationsd_3hrs"]);
        prRow.voltmean_24hrs = Convert.ToSingle(ooRow["voltmean_24hrs"]);
        prRow.rotatemean_24hrs = Convert.ToSingle(ooRow["rotatemean_24hrs"]);
        prRow.pressuremean_24hrs = Convert.ToSingle(ooRow["pressuremean_24hrs"]);
        prRow.vibrationmean_24hrs = Convert.ToSingle(ooRow["vibrationmean_24hrs"]);
        prRow.voltstd_24hrs = Convert.ToSingle(ooRow["voltsd_24hrs"]);
        prRow.rotatestd_24hrs = Convert.ToSingle(ooRow["rotatesd_24hrs"]);
        prRow.pressurestd_24hrs = Convert.ToSingle(ooRow["pressuresd_24hrs"]);
        prRow.vibrationstd_24hrs = Convert.ToSingle(ooRow["vibrationsd_24hrs"]);
        prRow.error1count = Convert.ToSingle(ooRow["error1count"]);
        prRow.error2count = Convert.ToSingle(ooRow["error2count"]);
        prRow.error3count = Convert.ToSingle(ooRow["error3count"]);
        prRow.error4count = Convert.ToSingle(ooRow["error4count"]);
        prRow.error5count = Convert.ToSingle(ooRow["error5count"]);
        prRow.sincelastcomp1 = Convert.ToSingle(ooRow["sincelastcomp1"]);
        prRow.sincelastcomp2 = Convert.ToSingle(ooRow["sincelastcomp2"]);
        prRow.sincelastcomp3 = Convert.ToSingle(ooRow["sincelastcomp3"]);
        prRow.sincelastcomp4 = Convert.ToSingle(ooRow["sincelastcomp4"]);
        prRow.model = (string)ooRow["model"];
        prRow.age = Convert.ToSingle(ooRow["age"]);
        prRow.failure = (string)ooRow["failure"];
        //
        return prRow;
    }));
            
    return dataView;
}

In [8]:
//Split dataset in two parts: TrainingDataset  and TestDataset          
var trainData = loadFromDataFrame(mlContext, trainDF);
var testData = loadFromDataFrame(mlContext, testDF);

Prior to start training we need to process that data, so that we encoded all non-numerical columns into numerical columns. Also we need to define which columns are going to be part of the `Features`and which one will be label. For this reason we define `PrepareData` method. 

In [9]:
public static IEstimator<ITransformer> PrepareData(MLContext mlContext)
{
    //one hot encoding category column
    IEstimator<ITransformer> dataPipeline =

    mlContext.Transforms.Conversion.MapValueToKey(outputColumnName: "Label", inputColumnName: nameof(PrMaintenanceClass.failure))
    //encode model column
    .Append(mlContext.Transforms.Categorical.OneHotEncoding("model",outputKind: OneHotEncodingEstimator.OutputKind.Indicator))

    //define features column
    .Append(mlContext.Transforms.Concatenate("Features",
    // 
    nameof(PrMaintenanceClass.voltmean_3hrs), nameof(PrMaintenanceClass.rotatemean_3hrs),
    nameof(PrMaintenanceClass.pressuremean_3hrs),nameof(PrMaintenanceClass.vibrationmean_3hrs),
    nameof(PrMaintenanceClass.voltstd_3hrs), nameof(PrMaintenanceClass.rotatestd_3hrs), 
    nameof(PrMaintenanceClass.pressurestd_3hrs), nameof(PrMaintenanceClass.vibrationstd_3hrs), 
    nameof(PrMaintenanceClass.voltmean_24hrs),nameof(PrMaintenanceClass.rotatemean_24hrs),
    nameof(PrMaintenanceClass.pressuremean_24hrs),nameof(PrMaintenanceClass.vibrationmean_24hrs), 
    nameof(PrMaintenanceClass.voltstd_24hrs),nameof(PrMaintenanceClass.rotatestd_24hrs),
    nameof(PrMaintenanceClass.pressurestd_24hrs),nameof(PrMaintenanceClass.vibrationstd_24hrs), 
    nameof(PrMaintenanceClass.error1count), nameof(PrMaintenanceClass.error2count),
    nameof(PrMaintenanceClass.error3count), nameof(PrMaintenanceClass.error4count), 
    nameof(PrMaintenanceClass.error5count), nameof(PrMaintenanceClass.sincelastcomp1),
    nameof(PrMaintenanceClass.sincelastcomp2),nameof(PrMaintenanceClass.sincelastcomp3),
    nameof(PrMaintenanceClass.sincelastcomp4),nameof(PrMaintenanceClass.model), nameof(PrMaintenanceClass.age) ));

    return dataPipeline;
}

As can be seen, the method converts the label column `failure` which is a simple textual column into categorical columns containing numerical representation for each different category called `Keys`.

Now that we have finished with data transformation, we are going to define the `Train` method which is going to implement ML algorithm, hyperparameters for it and training process. Once we call this method the method will return the trained model.

In [10]:
//train method
static public TransformerChain<ITransformer> Train(MLContext mlContext, IDataView preparedData)
{
    var transformationPipeline=PrepareData(mlContext);
    //settings hyper parameters
    var options = new LightGbmMulticlassTrainer.Options();
    options.FeatureColumnName = "Features";
    options.LearningRate = 0.005;
    options.NumberOfLeaves = 70;
    options.NumberOfIterations = 2000;
    options.NumberOfLeaves = 50;
    options.UnbalancedSets = true;
    //
    var boost = new DartBooster.Options();
    boost.XgboostDartMode = true;
    boost.MaximumTreeDepth = 25;
    options.Booster = boost;
    
    // Define LightGbm algorithm estimator
    IEstimator<ITransformer> lightGbm = mlContext.MulticlassClassification.Trainers.LightGbm(options);

    //train the ML model
    TransformerChain<ITransformer> model = transformationPipeline.Append(lightGbm).Fit(preparedData);

    //return trained model for evaluation
    return model;
}

# Training proces and model evaluation

Since we have all rquired methods, the main program structure looks like:


In [11]:
//prepare data transformation pipeline
var dataPipeline = PrepareData(mlContext);

//print prepared data
var pp = dataPipeline.Fit(trainData);
var transformedData = pp.Transform(trainData);

In [ ]:
//train the model
var model = Train(mlContext, trainData);

Once the `Train` method retrns the model, the evaluation phase started. In order to evaluate model, we performe full evaluation with trainig and testing data. 

# Model Evaluation with train data set

In [ ]:
//evaluate train set
var predictions = model.Transform(trainData);
var metricsTrain = mlContext.MulticlassClassification.Evaluate(predictions);

In [ ]:
ConsoleHelper.PrintMultiClassClassificationMetrics("TRAIN DataSet", metricsTrain);
ConsoleHelper.ConsoleWriteHeader("Train DataSet Confusion Matrix ");
ConsoleHelper.ConsolePrintConfusionMatrix(metricsTrain.ConfusionMatrix);


As can be seen the model predict the values corectly in most cases in the traini dataset. Now lets see how the model predict the data which have not been part of the raining process.

# Model evaluation with test data set

In [ ]:
//evaluate test set
var testPrediction = model.Transform(testData);
var metricsTest = mlContext.MulticlassClassification.Evaluate(testPrediction);
ConsoleHelper.PrintMultiClassClassificationMetrics("Test Dataset", metricsTest);

ConsoleHelper.ConsoleWriteHeader("Test DataSet Confusion Matrix ");
ConsoleHelper.ConsolePrintConfusionMatrix(metricsTest.ConfusionMatrix);

We can see, that the model has overall accuracy 99%, and 95% average per class accuracy.